In [53]:
!pip -q install datasets
!pip install spacy-universal-sentence-encoder
from datasets import load_dataset
import spacy_universal_sentence_encoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import spacy
import numpy as np

nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = load_dataset("PiC/phrase_similarity")

Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
def encode(examples):
    return tokenizer(examples["phrase1"], examples["phrase2"], truncation=True, padding="max_length")

dataset = dataset.map(encode, batched=True)

Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [34]:
def cosine_sim(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity

In [60]:
train_data = dataset["test"]
# boundary_score = 0.2
# print(len(train_data.keys()))
q = 0
count = 0
f = 0
t = 0
ok_score = 0
tp, tn, fp, fn = 0, 0, 0, 0
boundary_score = 0.5
for item in train_data:
    q += 1
    # print(item['phrase1'], item['phrase2'], item['label'])
    label = item['label']
    phrase1 = item['phrase1'].lower()
    phrase2 = item['phrase2'].lower()
    doc1 = nlp(phrase1)
    doc2 = nlp(phrase2)
    doc_sim = doc1.similarity(doc2)

    vec_arr1 = []
    vec_arr2 = []
    # if (len(doc1) != len(doc2)):
        # print(phrase1, phrase2, )
    pos1 = []
    pos2 = []
    # print(label)
    for token in doc1:
        pos1.append(token.pos_)
        vec_arr1.append(token.vector)
        # else:
            # print(phrase1, phrase2)
    for token in doc2:
        pos2.append(token.pos_)
        vec_arr2.append(token.vector)
    if (pos1 == pos2):
        sum = 0
        if (label == 1):
            t+=1
        else:
            f+=1
        # print("okokok")
        for i in range(len(vec_arr1)):
            sum += cosine_sim(vec_arr1[i], vec_arr2[i])
        sum /= len(vec_arr1)
        sim_score = sum
        # print(sim_score)

    else:
        if (vec_arr1):
            vec1sum = np.zeros_like(vec_arr1[0])
            vec2sum = np.zeros_like(vec_arr1[0])
            for vec in vec_arr1:
                vec1sum = vec1sum + vec
            for vec in vec_arr2:
                vec2sum = vec2sum + vec

            sim_score = cosine_sim(vec1sum, vec2sum)
            if (sim_score < 0):
                print(phrase1, phrase2)
                print(sim_score)
            # print(sim_score)
        else:
            print(phrase1, phrase2)
            print(pos1, pos2)
    # print(sim_score, doc_sim)
    if (sim_score > boundary_score):
        if (label == 1):
            tp += 1
        else:
            fp += 1
    else:
        if (label == 1):
            fn += 1
        else:
            tn += 1
# print(we)
print(ok_score)
print(t, f)
print((tp + tn)/(fp + fn + tp + tn))
# print(tp, tn, fp, fn)
# doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)


0
732 760
0.5135
791 236 764 209
